# Coffee Level Detection Batch Inference

This notebook runs inference on all images in a given directory using a trained coffeeCNN model and saves results to a JSON file matching the format of `compiled_coffee_level_annotations.json`.

In [ ]:
# 1. Import Required Libraries
import os
import json
from datetime import datetime
from tqdm import tqdm
from coffee_level_detection.inference.tools import infer_coffee_level


In [ ]:
# 2. Load Pre-trained Inference Model
image_dir = "processed_images"  # Change as needed
model_path = "coffee_level_detection/inference/coffeeCNN.pth"  # Change as needed

# No need to load model explicitly, infer_coffee_level loads it per call (can be optimized for batch)


In [ ]:
# 3. Run Inference on All Images in image_dir
results = []
annotator = "inference"
version = "1.0"

image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))]
print(f"Found {len(image_files)} images in {image_dir}")

for fname in tqdm(image_files, desc="Running inference"):
    img_path = os.path.join(image_dir, fname)
    try:
        coffee_level = infer_coffee_level(img_path, model_path)
    except Exception as e:
        print(f"Error processing {fname}: {e}")
        coffee_level = None
    results.append({
        "filename": fname,
        "coffee_level": coffee_level,
        "timestamp": datetime.now().isoformat(),
        "annotator": annotator,
        "version": version
    })


In [ ]:
# 4. Compile Results into JSON Format
output_json = {
    "total_annotated": len(results),
    "session_timestamp": datetime.now().isoformat(),
    "level_distribution": {},
    "annotation_data": results
}

# Compute level distribution
levels = [r["coffee_level"] for r in results if r["coffee_level"] is not None]
from collections import Counter
output_json["level_distribution"] = dict(Counter(levels))


In [ ]:
# 5. Save Results to JSON File
output_path = "inference_coffee_level_annotations.json"  # Change as needed
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output_json, f, indent=2)
print(f"Saved inference results to {output_path}")